## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-24-23-49-11 +0000,nyt,U.S. Attorney Scrambles to Present Case Agains...,https://www.nytimes.com/2025/09/24/us/politics...
1,2025-09-24-23-46-22 +0000,nyt,Judge Blocks Trump Administration’s Efforts to...,https://www.nytimes.com/2025/09/24/us/politics...
2,2025-09-24-23-44-41 +0000,nyt,Abdul Gani Bhat Dies at 93; Kashmiri Separatis...,https://www.nytimes.com/2025/09/24/world/asia/...
3,2025-09-24-23-44-00 +0000,nypost,Teachers’ union-backing NY pols who called for...,https://nypost.com/2025/09/24/us-news/union-ti...
4,2025-09-24-23-43-14 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/09/24/us/dal...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2308/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
9,trump,57
81,new,19
187,kimmel,19
75,ice,16
46,china,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
202,2025-09-24-14-05-20 +0000,nypost,Trump threatens to sue ABC over Jimmy Kimmel’s...,https://nypost.com/2025/09/24/us-news/trump-th...,123
298,2025-09-24-05-04-00 +0000,wsj,Kimmel Makes Emotional Return to ABC With Trum...,https://www.wsj.com/business/media/jimmy-kimme...,119
79,2025-09-24-21-25-02 +0000,nypost,Trump blames ‘Radical Left Democrats’ rhetoric...,https://nypost.com/2025/09/24/us-news/trump-bl...,111
270,2025-09-24-09-30-00 +0000,wsj,Stephen Miran set out his case for lower inter...,https://www.wsj.com/economy/central-banking/st...,107
66,2025-09-24-21-39-41 +0000,wapo,President Donald Trump wrote on TruthSocial th...,https://www.washingtonpost.com,107


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
202,123,2025-09-24-14-05-20 +0000,nypost,Trump threatens to sue ABC over Jimmy Kimmel’s...,https://nypost.com/2025/09/24/us-news/trump-th...
42,78,2025-09-24-22-20-12 +0000,nypost,"Vance tears into Democrats, calls out Gavin Ne...",https://nypost.com/2025/09/24/us-news/vance-te...
329,66,2025-09-24-00-42-57 +0000,cbc,Carney says he will meet China's president 'at...,https://www.cbc.ca/news/politics/carney-china-...
23,46,2025-09-24-22-55-47 +0000,nypost,Trump’s pro-Ukraine shift a ‘strategic move’ b...,https://nypost.com/2025/09/24/world-news/trump...
297,44,2025-09-24-05-11-13 +0000,nypost,Typhoon Ragasa batters Hong Kong and southern ...,https://nypost.com/2025/09/24/world-news/typho...
191,43,2025-09-24-15-06-31 +0000,bbc,Israeli forces near Gaza City centre as Hamas ...,https://www.bbc.com/news/articles/c4gk9dreldxo...
262,38,2025-09-24-10-00-00 +0000,nypost,Vivek Ramaswamy tells ‘Pod Force One’ US may h...,https://nypost.com/2025/09/24/us-news/vivek-ra...
117,35,2025-09-24-19-49-43 +0000,nypost,Union launches vicious $1M ad campaign against...,https://nypost.com/2025/09/24/us-news/union-la...
44,31,2025-09-24-22-18-27 +0000,nypost,Sisters who trashed Charlie Kirk memorial begg...,https://nypost.com/2025/09/24/us-news/arkansas...
38,31,2025-09-24-22-26-05 +0000,nypost,Murder suspect dodges the law for months — unt...,https://nypost.com/2025/09/24/us-news/murder-s...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
